# Face detection in Camera feed or video feed

We can reuse the code of module-1 to adjust to our needs

### Importing the libraries

In [14]:
import numpy as np
import cv2
from imutils.video import WebcamVideoStream
import imutils
import time

If an error 'module not found' occurs just do <br>
<b>pip install MODULE-NAME</b> in cmd

### Creating the model 

In [15]:
net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt', 'res10_300x300_ssd_iter_140000.caffemodel')

### Creating a Video stream object

Instead of reading an image we will use the webcam to capture images and loop throught them.

In [16]:
vs = WebcamVideoStream(src=0).start()
time.sleep(2.0)

### Looping throught the images from webcam and detecting faces

In [17]:
while True:
    # Reading a frame from the webcam
    frame = vs.read()
    
    # Resizing the image
    frame = imutils.resize(frame, width=400)
    
    # Getting Height and width of the image
    (h, w) = frame.shape[:2]
    
    # Pre-processing the image
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    
    # Setting it as input
    net.setInput(blob)
    
    # Passing the image through the network
    detections = net.forward()
    
    # Looping throught the predictions
    for i in range(detections.shape[2]):

        # Extract confidence from detection for the prediction
        confidence = detections[0, 0, i, 2]  

        # If prediction is greter than 0.5 (threshold)
        if confidence > 0.5: 

            # Get the bounding box coordinates
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])

            # Converting the coordinates to intergers
            (startX, startY, endX, endY) = box.astype("int") 

            # Drawing the bounding box around the detected object
            cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)

            # Formatting the confidence to 2 decimal point
            text = "{:.2f}%".format(confidence * 100)

            # Setting the coordinate for the confidence text
            y = startY - 10 if startY - 10 > 10 else startY + 10

            # Putting the text 
            cv2.putText(frame, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            
    # Displaying the image
    cv2.imshow("Output", frame)
    
    # Wait for a key press
    key = cv2.waitKey(1)
    
    # If the key pressed was q we will break from the loop
    if key == ord("q"):
        break


# Destorying all the windows
cv2.destroyAllWindows()

# Releasing the camera
vs.stop()

 There's a bug in the imutil library because of which you will not be able to release the webcam. To fix this edit the <b>"webcamvideostream.py"</b> file in <b>"Anaconda3/Lib/site-packages/imutils/video/webcamvideostream.py"</b> and in the stop method add the following line of code at the end and save it.
#  
self.stream.release()